### 📦 Package Installation

This code block installs the required dependencies for the telecommunications customer onboarding tracking system.

In [ ]:
# 📦 Install OpenAI Agents SDK and other dependencies
!pip install -q openai-agents

### 🔑 API Key Configuration

This code block securely handles the OpenAI API key setup required for the agents to communicate with OpenAI's models.


In [ ]:
import os
import getpass

# Prompt the user to enter the API key without showing it
api_key = getpass.getpass(prompt="Enter your OpenAI API key: ")

# Set the API key as an environment variable
os.environ['OPENAI_API_KEY'] = api_key

Enter your OpenAI API key: ··········


## 📚 Import Dependencies

This code block imports all the necessary libraries and modules required for the telecommunications customer onboarding tracking system.

In [ ]:
import re
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
from dataclasses import dataclass, field # Added dataclass and field
from typing import List, Dict, Optional
from agents import Agent, Runner, function_tool, RunContextWrapper

## 📱 Customer Onboarding Data Structure

This code block defines the core data structure that represents a customer's complete onboarding journey in the telecommunications system.

#### Core Structure

The `CustomerOnboarding` dataclass captures essential customer information and journey metrics:

##### Customer Information
- **Basic Details**: Customer ID, name, surname, email, and signup date
- **Service Details**: Customer type (individual/business/enterprise) and service type (mobile/internet/bundle)
- **Journey Status**: Activation date and current onboarding progress

##### Journey Tracking
- **Onboarding Steps**: Dictionary mapping step names to completion times in minutes
- **Failed Steps**: List of steps that encountered failures during the process
- **Support Engagement**: Count of support tickets raised during onboarding

##### Auto-Calculated Fields
- **Churn Probability Rate**: Automatically calculated risk score based on journey experience
- **Email Validation**: Boolean flag indicating email format validity

#### Churn Probability Algorithm

The system calculates churn risk using a sophisticated multi-factor approach:

##### Base Risk Assessment
- Starts with a 10% baseline churn probability
- Applies various risk multipliers based on journey friction points

##### Risk Factors Considered

**Time-Based Penalties:**
- Total onboarding time exceeding thresholds (90, 120, 180 minutes)
- Delayed activation beyond expected timeframes (24, 48, 72 hours)

**Process Friction:**
- Number of failed steps with escalating penalties
- Support ticket volume indicating customer frustration

**Step-Specific Performance:**
- Individual step completion times compared to expected thresholds
- Critical steps like identity verification, credit checks, and service activation have specific time expectations

**Customer Segmentation:**
- Enterprise customers receive 20% risk reduction (higher tolerance)
- Individual customers receive 10% risk increase (lower tolerance)

#### Key Features

##### Automated Processing
The `__post_init__` method ensures that churn probability and email validation are automatically calculated upon object creation, eliminating manual computation requirements.

##### Comprehensive Risk Modeling
The churn calculation considers multiple dimensions of the customer experience, from technical process efficiency to customer support interactions, providing a holistic view of retention risk.

##### Flexible Step Tracking
The onboarding steps dictionary allows for dynamic tracking of any number of process steps, making the system adaptable to different service types and onboarding workflows.

This data structure enables telecom companies to identify at-risk customers early in their journey and implement targeted retention strategies based on specific friction points in the onboarding process.

In [ ]:
# 📱 Enhanced structured input for customer onboarding journey
@dataclass
class CustomerOnboarding:
    customer_id: str
    name: str
    surname: str
    email: str
    signup_date: str
    activation_date: Optional[str]  # None if not activated yet
    onboarding_steps: Dict[str, int]  # step_name: completion_time_in_minutes
    failed_steps: List[str]  # list of steps that failed
    customer_type: str  # "individual", "business", "enterprise"
    service_type: str  # "mobile", "internet", "bundle"
    support_tickets: int  # number of support tickets during onboarding
    churn_probability_rate: float = field(init=False)  # Auto-calculated based on journey
    email_valid: bool = field(init=False)  # Auto-validated

    def __post_init__(self):
        """Calculate churn probability and validate email after initialization"""
        self.churn_probability_rate = self._calculate_churn_probability()
        self.email_valid = self._validate_email_format()

    def _calculate_churn_probability(self) -> float:
        """Calculate churn probability based on onboarding experience"""
        base_risk = 0.1  # 10% base churn rate
        risk_factors = 0.0

        # Time-based risk factors
        total_onboarding_time = sum(self.onboarding_steps.values())
        if total_onboarding_time > 180:  # More than 3 hours
            risk_factors += 0.3
        elif total_onboarding_time > 120:  # More than 2 hours
            risk_factors += 0.2
        elif total_onboarding_time > 90:  # More than 1.5 hours
            risk_factors += 0.1

        # Activation delay risk
        if self.activation_date:
            signup = datetime.fromisoformat(self.signup_date)
            activation = datetime.fromisoformat(self.activation_date)
            hours_to_activate = (activation - signup).total_seconds() / 3600

            if hours_to_activate > 72:  # More than 3 days
                risk_factors += 0.25
            elif hours_to_activate > 48:  # More than 2 days
                risk_factors += 0.15
            elif hours_to_activate > 24:  # More than 1 day
                risk_factors += 0.1
        else:
            # Not activated yet - high risk
            risk_factors += 0.4

        # Failed steps risk
        failed_count = len(self.failed_steps)
        if failed_count >= 3:
            risk_factors += 0.3
        elif failed_count >= 2:
            risk_factors += 0.2
        elif failed_count >= 1:
            risk_factors += 0.1

        # Support tickets risk
        if self.support_tickets > 5:
            risk_factors += 0.25
        elif self.support_tickets > 3:
            risk_factors += 0.15
        elif self.support_tickets > 1:
            risk_factors += 0.1

        # Step-specific time penalties
        step_thresholds = {
            "identity_verification": 10,
            "credit_check": 15,
            "plan_selection": 20,
            "equipment_provisioning": 45,
            "service_activation": 30,
            "account_setup": 15
        }

        for step, time_taken in self.onboarding_steps.items():
            if step in step_thresholds:
                threshold = step_thresholds[step]
                if time_taken > threshold * 2:  # Double the expected time
                    risk_factors += 0.1
                elif time_taken > threshold * 1.5:  # 50% over expected time
                    risk_factors += 0.05

        # Customer type adjustments
        if self.customer_type == "enterprise":
            risk_factors *= 0.8  # Enterprise customers slightly more tolerant
        elif self.customer_type == "individual":
            risk_factors *= 1.1  # Individual customers less tolerant

        total_risk = min(base_risk + risk_factors, 0.95)  # Cap at 95%
        return round(total_risk, 3)

    def _validate_email_format(self) -> bool:
        """Basic email format validation"""
        email_pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
        return bool(re.match(email_pattern, self.email))



## 📱 Customer Onboarding Analysis Functions

### 1. Email Validation Tool
**Purpose**: Checks if the customer's email address is legitimate and trustworthy

**How it works**:
- Starts with basic format checking (already done automatically)
- Gives the email a score out of 100 points
- **Adds points for**: Popular domains (Gmail, Yahoo, etc.), business domains, emails that match the customer's name
- **Removes points for**: Suspicious patterns (like double dots), uncommon domains, emails that don't match the person's name

**Results**:
- ✅ **VALID** (85+ points): Email looks good
- 🟡 **QUESTIONABLE** (70-84 points): Email might have issues
- ⚠️ **SUSPICIOUS** (below 70): Email looks risky

### 2. Bottleneck Detection Tool
**Purpose**: Finds where customers are getting stuck in the onboarding process

**Expected timeframes**:
- Identity verification: 10 minutes
- Credit check: 15 minutes
- Plan selection: 20 minutes
- Equipment setup: 45 minutes
- Service activation: 30 minutes
- Account setup: 15 minutes

**Results**:
- Lists any steps that took too long
- Shows which steps completely failed
- Confirms if everything went smoothly

### 3. Activation Status Checker
**Purpose**: Tracks how quickly customers activated their service

**Timing benchmarks**:
- ✅ **Fast**: Under 24 hours
- 🟡 **Moderate**: 24-72 hours (1-3 days)
- ⚠️ **Slow**: Over 72 hours (3+ days)
- 🚨 **Not activated**: Still waiting

### 4. Churn Risk Assessment
**Purpose**: Predicts how likely a customer is to cancel their service

**Risk levels**:
- 🚨 **CRITICAL** (80%+ risk): Customer will probably leave - act now!
- ⚠️ **HIGH** (60-79%): Customer frustrated - needs priority help
- 🟡 **MODERATE** (40-59%): Keep an eye on this customer
- 🟢 **LOW** (20-39%): Customer doing fine
- ✅ **MINIMAL** (under 20%): Happy customer

**What increases risk**:
- Too many support tickets
- Multiple failed steps
- Taking too long to complete onboarding
- Not activating service yet

These functions help customer service teams quickly understand:
- Which customers need immediate help
- What specific problems they're facing
- How to prioritize their workload
- Whether a customer might cancel their service

In [ ]:
# 🔧 Enhanced function tools for analysis
@function_tool
def validate_customer_email(wrapper: RunContextWrapper[CustomerOnboarding]) -> str:
    """Validate customer email using advanced agentic validation"""
    customer = wrapper.context

    # Basic format validation
    if not customer.email_valid:
        return f"❌ INVALID EMAIL FORMAT: '{customer.email}' - Basic format validation failed"

    # Advanced validation checks
    email_parts = customer.email.split('@')
    domain = email_parts[1] if len(email_parts) == 2 else ""

    # Common domain validation
    trusted_domains = [
        'gmail.com', 'yahoo.com', 'outlook.com', 'hotmail.com', 'aol.com',
        'icloud.com', 'protonmail.com', 'mail.com', 'zoho.com'
    ]

    business_indicators = ['.com', '.org', '.net', '.edu', '.gov', '.mil']

    validation_score = 100
    issues = []

    # Check for suspicious patterns
    if '..' in customer.email:
        validation_score -= 30
        issues.append("consecutive dots detected")

    if customer.email.startswith('.') or customer.email.endswith('.'):
        validation_score -= 20
        issues.append("starts/ends with dot")

    # Domain reputation check (simplified)
    if domain in trusted_domains:
        validation_score += 10
    elif any(domain.endswith(indicator) for indicator in business_indicators):
        validation_score += 5
    else:
        validation_score -= 10
        issues.append("uncommon domain")

    # Name consistency check
    email_prefix = email_parts[0].lower()
    full_name = f"{customer.name.lower()}{customer.surname.lower()}"
    name_parts = [customer.name.lower(), customer.surname.lower()]

    if any(part in email_prefix for part in name_parts) or full_name in email_prefix.replace('.', ''):
        validation_score += 15
    else:
        validation_score -= 5
        issues.append("email doesn't match name")

    # Final validation result
    if validation_score >= 85:
        status = "✅ VALID"
        confidence = "HIGH"
    elif validation_score >= 70:
        status = "🟡 QUESTIONABLE"
        confidence = "MEDIUM"
    else:
        status = "⚠️ SUSPICIOUS"
        confidence = "LOW"

    result = f"{status} EMAIL: '{customer.email}' (Email Validation: {status}, Confidence: {confidence})"
    if issues:
        result += f" | Issues: {', '.join(issues)}"

    return result

@function_tool
def analyze_onboarding_bottlenecks(wrapper: RunContextWrapper[CustomerOnboarding]) -> str:
    """Identify bottlenecks in the onboarding process"""
    steps = wrapper.context.onboarding_steps
    failed = wrapper.context.failed_steps

    # Define acceptable time thresholds for each step (in minutes)
    thresholds = {
        "identity_verification": 10,
        "credit_check": 15,
        "plan_selection": 20,
        "equipment_provisioning": 45,
        "service_activation": 30,
        "account_setup": 15
    }

    slow_steps = []
    for step, time_taken in steps.items():
        if step in thresholds and time_taken > thresholds[step]:
            slow_steps.append(f"{step}: {time_taken}min (threshold: {thresholds[step]}min)")

    result = []
    if slow_steps:
        result.append(f"⚠️ Slow steps detected: {'; '.join(slow_steps)}")

    if failed:
        result.append(f"❌ Failed steps: {', '.join(failed)}")

    if not result:
        result.append("✅ No major bottlenecks detected in onboarding process")

    return " | ".join(result)

@function_tool
def check_activation_status(wrapper: RunContextWrapper[CustomerOnboarding]) -> str:
    """Check if customer successfully activated and calculate time to activation"""
    customer = wrapper.context

    if customer.activation_date is None:
        return "🚨 Customer not yet activated - requires immediate attention"

    # Calculate time from signup to activation
    signup = datetime.fromisoformat(customer.signup_date)
    activation = datetime.fromisoformat(customer.activation_date)
    time_to_activate = (activation - signup).total_seconds() / 3600  # hours

    if time_to_activate > 72:  # More than 3 days
        return f"⚠️ Slow activation: {time_to_activate:.1f} hours (Target: <72h)"
    elif time_to_activate > 24:  # More than 1 day
        return f"🟡 Moderate activation time: {time_to_activate:.1f} hours"
    else:
        return f"✅ Fast activation: {time_to_activate:.1f} hours"

@function_tool
def assess_churn_risk(wrapper: RunContextWrapper[CustomerOnboarding]) -> str:
    """Assess churn risk based on automatically calculated probability"""
    customer = wrapper.context
    churn_prob = customer.churn_probability_rate

    # Categorize risk levels
    if churn_prob >= 0.8:
        risk_level = "🚨 CRITICAL RISK"
        recommendation = "Immediate retention team intervention required"
    elif churn_prob >= 0.6:
        risk_level = "⚠️ HIGH RISK"
        recommendation = "Priority support and proactive outreach needed"
    elif churn_prob >= 0.4:
        risk_level = "🟡 MODERATE RISK"
        recommendation = "Enhanced monitoring and automated check-ins"
    elif churn_prob >= 0.2:
        risk_level = "🟢 LOW RISK"
        recommendation = "Standard onboarding process sufficient"
    else:
        risk_level = "✅ MINIMAL RISK"
        recommendation = "Excellent onboarding experience"

    # Identify contributing factors that led to this calculation
    risk_factors = []
    if customer.support_tickets > 3:
        risk_factors.append(f"High support tickets: {customer.support_tickets}")

    if len(customer.failed_steps) > 1:
        risk_factors.append(f"Multiple failed steps: {len(customer.failed_steps)}")

    if customer.activation_date is None:
        risk_factors.append("Activation not completed")

    total_time = sum(customer.onboarding_steps.values())
    if total_time > 120:
        risk_factors.append(f"Long onboarding time: {total_time}min")

    result = f"{risk_level} ({churn_prob:.1%} probability - AUTO-CALCULATED) - {recommendation}"
    if risk_factors:
        result += f" | Contributing factors: {'; '.join(risk_factors)}"

    return result

## 📧 Generate Onboarding Email Function

Creates personalized email templates based on customer onboarding status, risk profile, and journey experience to improve engagement and retention.

**Email Classification Logic**:
The function uses a hierarchical decision tree to determine the most appropriate email type:

1. **Incomplete Onboarding** (Priority 1): Customer hasn't activated service
2. **High Risk Retention** (≥60% churn probability): Critical intervention needed
3. **Moderate Risk Support** (≥40% churn probability): Proactive support offering
4. **Recovery Acknowledgment**: Failed steps or high support tickets (>2) detected
5. **Welcome Success**: Default for smooth onboarding experiences

**Email Template Structure**:
Each template includes:
- **Dynamic Subject Line**: Personalized with customer ID and service type
- **Tone Specification**: Guides the emotional approach (celebratory, supportive, concerned, etc.)
- **Key Points Array**: Structured content guidelines for email body
- **Customer Context**: Relevant data points for personalization

**Template Categories**:

### Welcome Success
- **Trigger**: Smooth onboarding, low churn risk (<40%), minimal issues
- **Tone**: Celebratory and welcoming
- **Focus**: Service benefits, quick start guidance, account management

### Recovery Acknowledgment  
- **Trigger**: Failed steps present OR support tickets >2, but activation completed
- **Tone**: Understanding and supportive
- **Focus**: Acknowledging challenges, confirming resolution, offering ongoing support

### Moderate Risk Support
- **Trigger**: 40-59% churn probability
- **Tone**: Helpful and proactive
- **Focus**: Experience optimization, additional resources, feedback collection

### High Risk Retention
- **Trigger**: ≥60% churn probability
- **Tone**: Concerned and solution-focused
- **Focus**: Issue acknowledgment, immediate assistance, escalation options, service credits

### Incomplete Onboarding
- **Trigger**: No activation date (highest priority)
- **Tone**: Encouraging and supportive
- **Focus**: Setup completion assistance, step-by-step guidance, direct support

**Output Format**:
The function returns a structured template specification containing:
- Email classification type
- Personalized subject line with customer details
- Recommended tone for content creation
- Customer context summary (service type, risk metrics, support history)
- Bulleted key points for content development

In [ ]:
# 📧 Enhanced email generation function
@function_tool
def generate_onboarding_email(wrapper: RunContextWrapper[CustomerOnboarding]) -> str:
    """Generate personalized onboarding email based on customer journey"""
    customer = wrapper.context
    churn_prob = customer.churn_probability_rate

    # Determine email type based on onboarding status and risk
    if customer.activation_date is None:
        email_type = "incomplete_onboarding"
    elif churn_prob >= 0.6:
        email_type = "high_risk_retention"
    elif churn_prob >= 0.4:
        email_type = "moderate_risk_support"
    elif len(customer.failed_steps) > 0 or customer.support_tickets > 2:
        email_type = "recovery_acknowledgment"
    else:
        email_type = "welcome_success"

    # Email templates based on customer situation
    email_templates = {
        "welcome_success": {
            "subject": f"🎉 Welcome to FullCode Telecom, {customer.name}! Your {customer.service_type.title()} Service is Ready",
            "tone": "celebratory and welcoming",
            "key_points": [
                f"Address customer by name: {customer.name} {customer.surname}",
                "Congratulate on successful activation",
                "Highlight service benefits",
                "Provide quick start guide",
                "Share customer support info",
                "Include account management tips"
            ]
        },
        "recovery_acknowledgment": {
            "subject": f"We're Here to Help, {customer.name} - Your {customer.service_type.title()} Service Setup",
            "tone": "understanding and supportive",
            "key_points": [
                f"Personal greeting: {customer.name} {customer.surname}",
                "Acknowledge any setup challenges",
                "Confirm service is now working",
                "Offer continued support",
                "Provide helpful resources",
                "Include direct contact for assistance"
            ]
        },
        "moderate_risk_support": {
            "subject": f"Getting the Most from Your Service, {customer.name}",
            "tone": "helpful and proactive",
            "key_points": [
                f"Personal touch: {customer.name}",
                "Check in on service experience",
                "Offer optimization tips",
                "Provide additional resources",
                "Invite feedback",
                "Highlight support options"
            ]
        },
        "high_risk_retention": {
            "subject": f"Let's Make Your Experience Better, {customer.name}",
            "tone": "concerned and solution-focused",
            "key_points": [
                f"Personal attention: {customer.name} {customer.surname}",
                "Acknowledge service issues",
                "Offer immediate assistance",
                "Provide escalation contact",
                "Include service credits if applicable",
                "Request feedback call"
            ]
        },
        "incomplete_onboarding": {
            "subject": f"Complete Your Setup, {customer.name} - We're Here to Help",
            "tone": "encouraging and supportive",
            "key_points": [
                f"Personal encouragement: {customer.name}",
                "Acknowledge incomplete setup",
                "Offer assistance to complete",
                "Provide step-by-step guidance",
                "Include direct support contact",
                "Set expectations for next steps"
            ]
        }
    }

    template = email_templates[email_type]

    return f"""
EMAIL TYPE: {email_type.upper()}
RECIPIENT: {customer.name} {customer.surname} <{customer.email}>
EMAIL VALIDATION: {'✅ Valid' if customer.email_valid else '❌ Invalid'}
SUBJECT: {template['subject']}
TONE: {template['tone']}

CUSTOMER CONTEXT:
- Name: {customer.name} {customer.surname}
- Email: {customer.email}
- Service: {customer.service_type} | Type: {customer.customer_type}
- Churn Risk: {churn_prob:.1%} (AUTO-CALCULATED)
- Support Tickets: {customer.support_tickets}
- Failed Steps: {customer.failed_steps if customer.failed_steps else 'None'}
- Activation Status: {'Completed' if customer.activation_date else 'Incomplete'}
- Total Onboarding Time: {sum(customer.onboarding_steps.values())} minutes

KEY POINTS TO INCLUDE:
{chr(10).join([f'• {point}' for point in template['key_points']])}
"""


## 🤖 Specialized AI Agents Definition

This system defines a comprehensive multi-agent architecture for analyzing and improving customer onboarding processes in telecommunications.

### Agent Architecture Overview

#### 1. **Email Validation Agent**
- **Job**: Check if customer email addresses are legitimate
- **Model**: GPT-4o-mini (fast and efficient)
- **Tool**: Uses the email validation function we saw earlier
- **Focus**: Format checking, domain reputation, name matching

#### 2. **Bottleneck Analysis Agent**  
- **Job**: Find where customers get stuck in the onboarding process
- **Model**: GPT-4o (more powerful for complex analysis)
- **Tool**: Uses the bottleneck detection function
- **Focus**: Identifies slow steps and complete failures

#### 3. **Activation Agent**
- **Job**: Track how quickly customers activate their service
- **Model**: GPT-4o-mini (straightforward calculations)
- **Tool**: Uses the activation status checker
- **Focus**: Time-to-activation metrics and non-activated customers

#### 4. **Churn Risk Agent**
- **Job**: Predict which customers might cancel their service
- **Model**: GPT-4o-mini (works with pre-calculated risk scores)
- **Tool**: Uses the churn risk assessment function
- **Focus**: Analyzes the automatically calculated churn probability

#### 5. **Recommendation Agent**
- **Job**: Suggest actions to improve the customer experience
- **Model**: GPT-4o-mini (generates advice)
- **Tools**: None (synthesizes information from other agents)
- **Focus**: Creates actionable improvement suggestions

#### 6. **Email Agent** 📧
- **Job**: Write personalized emails for customers
- **Model**: GPT-4o (needs creativity and language skills)
- **Tool**: Uses the email generation function
- **Special Features**:
  - Has detailed instructions for writing professional emails
  - Uses specific company information (Alice AI from FullCode Telecom)
  - Adapts tone based on customer situation
  - Includes contact details and next steps

#### 🔄 How They Work Together

1. **Email Validation Agent** validates the customer's contact information
2. **Bottleneck Agent** identifies where they struggled
3. **Activation Agent** checks their completion status
4. **Churn Risk Agent** predicts if they might leave
5. **Recommendation Agent** suggests what to do next
6. **Email Agent** crafts the perfect follow-up message


In [ ]:
# 🤖 Enhanced specialized agents
email_validation_agent = Agent[CustomerOnboarding](
    name="EmailValidationAgent",
    instructions="Validate customer email addresses using comprehensive checks including format validation, domain reputation, and name consistency analysis.",
    model="gpt-4o-mini",
    tools=[validate_customer_email],
    output_type=str
)

bottleneck_agent = Agent[CustomerOnboarding](
    name="BottleneckAnalysisAgent",
    instructions="Analyze the customer onboarding journey to identify process bottlenecks and failed steps. Focus on steps that took longer than expected or failed completely.",
    model="gpt-4o",
    tools=[analyze_onboarding_bottlenecks],
    output_type=str
)

activation_agent = Agent[CustomerOnboarding](
    name="ActivationAgent",
    instructions="Check customer activation status and calculate time-to-activation metrics. Identify customers who haven't activated or took too long to activate.",
    model="gpt-4o-mini",
    tools=[check_activation_status],
    output_type=str
)

churn_risk_agent = Agent[CustomerOnboarding](
    name="ChurnRiskAgent",
    instructions="Assess customer churn risk based on the automatically calculated churn probability that considers onboarding experience, support tickets, failed steps, and overall journey quality.",
    model="gpt-4o",
    tools=[assess_churn_risk],
    output_type=str
)

recommendation_agent = Agent(
    name="OnboardingRecommendationAgent",
    instructions="Generate actionable recommendations to improve customer onboarding based on bottleneck analysis, activation status, churn risk assessment, and email validation results.",
    model="gpt-4o-mini",
    output_type=str
)

email_agent = Agent[CustomerOnboarding](
    name="OnboardingEmailAgent",
    instructions="""Generate personalized onboarding emails for telecommunications customers based on their journey experience.

Create professional, empathetic emails that:
1. Address the customer by their full name (name + surname)
2. Match the appropriate tone for the customer's situation
3. Address specific issues they encountered
4. Provide relevant next steps and resources
5. Include appropriate urgency level based on auto-calculated churn risk
6. Maintain brand voice while being genuinely helpful
7. Consider email validation status when determining communication strategy

Replace the following fields accordingly:
[Your Name]: Alice AI
[Your Position]: Customer Relations Specialist
[Company Name]: FullCode Telecom Inc.
[Contact Number]: +1234567890
[Contact Email]: alice@fullcodetelecom.com

Use the customer context provided to personalize the message and ensure all key points are covered naturally in the email content.""",
    model="gpt-4o-mini",
    tools=[generate_onboarding_email],
    output_type=str
)

# Customer Onboarding Analysis Pipeline

This is the main function that orchestrates all six AI agents to perform a comprehensive analysis of a customer's onboarding journey. It's like a well-coordinated assembly line where each agent performs their specialized task in sequence.

## 🔄 Pipeline Overview

The function takes a `CustomerOnboarding` object (containing all customer data) and runs it through six sequential analysis steps, with each step building on the previous ones.

## 📋 Step-by-Step Process

### **Initial Setup**
- Displays customer basic information (name, email, service type)
- Shows the **auto-calculated churn risk** percentage
- Creates a visual separator for clean output

### **Step 1: Email Validation** 📧
- **Agent**: Email Validation Agent
- **Task**: Verify if the customer's email address is legitimate
- **Output**: Email validity status with confidence level and any issues found

### **Step 2: Bottleneck Analysis** 🔍
- **Agent**: Bottleneck Analysis Agent  
- **Task**: Identify where the customer got stuck during onboarding
- **Output**: List of slow steps, failed processes, or confirmation that everything went smoothly

### **Step 3: Activation Check** 🔍
- **Agent**: Activation Agent
- **Task**: Verify service activation status and timing
- **Output**: Activation timeline and whether it meets acceptable standards

### **Step 4: Churn Risk Assessment** 🔍
- **Agent**: Churn Risk Agent
- **Task**: Analyze the pre-calculated churn probability and contributing factors
- **Output**: Risk level categorization with specific recommendations

### **Step 5: Business Recommendations** 🧠
- **Agent**: Recommendation Agent
- **Task**: Synthesize all previous results into actionable business advice
- **Input**: Results from all four previous analyses
- **Output**: Specific recommendations to improve the customer experience

### **Step 6: Personalized Email** 📧
- **Agent**: Email Agent
- **Task**: Create a custom email based on the customer's specific journey
- **Output**: Complete email with appropriate tone, subject line, and content

## 🔧 Technical Features

### **Asynchronous Processing**
- Uses `async/await` for efficient processing
- Can handle multiple customers simultaneously without blocking

### **Sequential Dependencies**
- Each step builds on previous results
- The recommendation agent uses outputs from steps 1-4
- The email agent considers the entire customer context

### **Comprehensive Output**
- Returns all six analysis results as a list
- Provides both console display and structured data return
- Clean formatting with visual separators and emojis

### **Context Sharing**
- Each agent receives the full customer data object
- Agents can access all customer journey information
- Recommendation agent gets compiled results from other agents

In [ ]:
# 🚀 Enhanced main pipeline function
async def run_onboarding_analysis(customer_journey: CustomerOnboarding):
    """Run complete onboarding analysis pipeline with email validation and generation"""
    print(f"📊 Analyzing onboarding for: {customer_journey.name} {customer_journey.surname} (ID: {customer_journey.customer_id})")
    print(f"Email: {customer_journey.email} | Service: {customer_journey.service_type} | Type: {customer_journey.customer_type}")
    print(f"🎯 Auto-calculated Churn Risk: {customer_journey.churn_probability_rate:.1%}")
    print("-" * 80)

    print("📧 Step 1: Email Validation Agent | Email Validation...")
    email_validation_result = await Runner.run(
        email_validation_agent,
        input=f"Validate email address for customer {customer_journey.name} {customer_journey.surname}",
        context=customer_journey
    )
    print(f"\t{email_validation_result.final_output}")

    print("\n🔍 Step 2: Bottleneck Agent | Bottleneck Analysis...")
    bottleneck_result = await Runner.run(
        bottleneck_agent,
        input=f"Analyze onboarding bottlenecks for customer {customer_journey.name} {customer_journey.surname}",
        context=customer_journey
    )
    print(f"\t{bottleneck_result.final_output}")

    print("\n🔍 Step 3: Activation Agent | Activation Status Check...")
    activation_result = await Runner.run(
        activation_agent,
        input=f"Check activation status for customer {customer_journey.name} {customer_journey.surname}",
        context=customer_journey
    )
    print(f"\t{activation_result.final_output}")

    print("\n🔍 Step 4: Churn Risk Agent | Churn Risk Assessment...")
    churn_result = await Runner.run(
        churn_risk_agent,
        input=f"Assess auto-calculated churn risk for customer {customer_journey.name} {customer_journey.surname}",
        context=customer_journey
    )
    print(f"\t{churn_result.final_output}")

    print("\n🧠 Step 5: Reccomendation Agent | Generating Final Recommendations...")
    recommendation_result = await Runner.run(
        recommendation_agent,
        input="Based on the analysis results including email validation, provide specific recommendations to improve the onboarding process and reduce churn risk",
        context=[email_validation_result.final_output, bottleneck_result.final_output, activation_result.final_output, churn_result.final_output]
    )
    print(f"\n✅ BUSINESS RECOMMENDATIONS:")
    print(recommendation_result.final_output)
    print("=" * 80)

    print("\n📧 Step 6: Email Agent | Generating Personalized Email...")
    email_result = await Runner.run(
        email_agent,
        input=f"Create personalized onboarding email for {customer_journey.name} {customer_journey.surname} based on their journey experience and auto-calculated churn risk",
        context=customer_journey
    )
    print(f"\t📬 Email Generated:")
    print(f"\t{email_result.final_output}")

    onboarding_result = [email_validation_result.final_output, bottleneck_result.final_output, activation_result.final_output, churn_result.final_output, recommendation_result.final_output, email_result.final_output]

    return onboarding_result


### Example Analysis: Successful Customer Onboarding

This example demonstrates the pipeline analysis for a well-executed customer onboarding journey. Let me break down what this successful case would reveal through each analysis step.

In [ ]:
# Example 1: Successful customer with good experience
successful_customer = CustomerOnboarding(
    customer_id="CUST_001",
    name="Nick",
    surname="Allister",
    email="nick.allister@gmail.com",
    signup_date="2025-06-25T09:00:00",
    activation_date="2025-06-25T15:30:00",
    onboarding_steps={
        "identity_verification": 8,
        "credit_check": 12,
        "plan_selection": 15,
        "equipment_provisioning": 35,
        "service_activation": 25,
        "account_setup": 10
    },
    failed_steps=[],
    customer_type="individual",
    service_type="mobile",
    support_tickets=1
)
print("🌟 SUCCESSFUL CUSTOMER ANALYSIS:")
result_1 = await run_onboarding_analysis(successful_customer)

🌟 SUCCESSFUL CUSTOMER ANALYSIS:
📊 Analyzing onboarding for: Nick Allister (ID: CUST_001)
Email: nick.allister@gmail.com | Service: mobile | Type: individual
🎯 Auto-calculated Churn Risk: 21.0%
--------------------------------------------------------------------------------
📧 Step 1: Email Validation Agent | Email Validation...
	The email address for customer Nick Allister, **nick.allister@gmail.com**, has been successfully validated. It is valid with high confidence.

🔍 Step 2: Bottleneck Agent | Bottleneck Analysis...
	The analysis of the onboarding process for customer Nick Allister revealed no major bottlenecks or failed steps. Everything seems to be running smoothly. If you need further insights or have specific concerns, feel free to ask!

🔍 Step 3: Activation Agent | Activation Status Check...
	Customer Nick Allister has successfully activated their account, taking 6.5 hours for activation, which is considered a fast activation. If you need further assistance or information about

### Example Analysis: Problematic Customer Onboarding

This example demonstrates the pipeline analysis for a severely compromised customer onboarding experience. This case illustrates how the system identifies critical issues and generates urgent intervention strategies.

In [ ]:
# Example 2: High-risk customer with problematic journey
problematic_customer = CustomerOnboarding(
    customer_id="CUST_002",
    name="Sarah",
    surname="Johnson",
    email="nemoask124@tempmail.com",
    signup_date="2025-06-20T10:00:00",
    activation_date=None,  # Not activated yet
    onboarding_steps={
        "identity_verification": 25,  # Slow
        "credit_check": 45,  # Very slow
        "plan_selection": 60,  # Extremely slow
        "equipment_provisioning": 90,  # Double expected time
        "service_activation": 0,  # Not completed
        "account_setup": 0  # Not completed
    },
    failed_steps=["credit_check", "service_activation", "account_setup"],
    customer_type="individual",
    service_type="internet",
    support_tickets=8
)

print("⚠️ HIGH-RISK CUSTOMER ANALYSIS:")
result_2 = await run_onboarding_analysis(problematic_customer)

⚠️ HIGH-RISK CUSTOMER ANALYSIS:
📊 Analyzing onboarding for: Sarah Johnson (ID: CUST_002)
Email: nemoask124@tempmail.com | Service: internet | Type: individual
🎯 Auto-calculated Churn Risk: 95.0%
--------------------------------------------------------------------------------
📧 Step 1: Email Validation Agent | Email Validation...
	The email address `nemoask124@tempmail.com` is valid with high confidence. However, there is an issue: it does not match the name "Sarah Johnson."

🔍 Step 2: Bottleneck Agent | Bottleneck Analysis...
	Here's the analysis for customer Sarah Johnson's onboarding journey:

### Slow Steps:
1. **Identity Verification**
   - Took 25 minutes (Threshold: 10 minutes)

2. **Credit Check**
   - Took 45 minutes (Threshold: 15 minutes)

3. **Plan Selection**
   - Took 60 minutes (Threshold: 20 minutes)

4. **Equipment Provisioning**
   - Took 90 minutes (Threshold: 45 minutes)

### Failed Steps:
1. **Credit Check**
2. **Service Activation**
3. **Account Setup**

These step

### Example Analysis: Incomplete Customer Onboarding

This example demonstrates the pipeline analysis for an abandoned onboarding journey. This case shows how the system identifies dropout points and generates re-engagement strategies for customers who haven't completed their activation.

In [ ]:
# Example 3: Incomplete onboarding customer
incomplete_customer = CustomerOnboarding(
    customer_id="CUST_003",
    name="Maria",
    surname="Tanova",
    email="maria.tanova@yahoo.com",
    signup_date="2025-06-26T14:00:00",
    activation_date=None,  # Not activated
    onboarding_steps={
        "identity_verification": 15,
        "credit_check": 20,
        "plan_selection": 45  # Stopped here
    },
    failed_steps=["equipment_provisioning"],
    customer_type="individual",
    service_type="internet",
    support_tickets=3
)
print("📋 INCOMPLETE ONBOARDING CUSTOMER ANALYSIS:")
result_3 = await run_onboarding_analysis(incomplete_customer)

📋 INCOMPLETE ONBOARDING CUSTOMER ANALYSIS:
📊 Analyzing onboarding for: Maria Tanova (ID: CUST_003)
Email: maria.tanova@yahoo.com | Service: internet | Type: individual
🎯 Auto-calculated Churn Risk: 87.0%
--------------------------------------------------------------------------------
📧 Step 1: Email Validation Agent | Email Validation...
	The email address for customer Maria Tanova, **maria.tanova@yahoo.com**, has been validated successfully. It is considered valid with high confidence.

🔍 Step 2: Bottleneck Agent | Bottleneck Analysis...
	In the onboarding journey for customer Maria Tanova, several bottlenecks were identified:

### Slow Steps:
1. **Identity Verification**: Took 15 minutes (threshold is 10 minutes).
2. **Credit Check**: Took 20 minutes (threshold is 15 minutes).
3. **Plan Selection**: Took 45 minutes (threshold is 20 minutes).

### Failed Steps:
- **Equipment Provisioning**: This step failed completely.

These areas may require further investigation and improvement to 

### 📤 Email Report via MCPTool and Zapier Gmail

We will implement the usage of MCP tools
- A report agent is called dynamically to generate a system summary
- The output is sent as an email using `MCPServerSSE` (which calls a Zapier Gmail webhook)

In [ ]:
from agents.mcp import MCPServer, MCPServerSse
from agents.model_settings import ModelSettings

In [ ]:
from agents import function_tool

@function_tool(strict_mode=False)
def synthesize_report(onboarding_result) -> str:
    """Synthesize the onboarding analysis results into a formatted report"""

    email_validation = onboarding_result[0]
    bottleneck_result = onboarding_result[1]
    activation_result = onboarding_result[2] # Added activation result
    churn_risk_result = onboarding_result[3]
    recommendation_result = onboarding_result[4] # Corrected index for recommendation

    # 📧 Compose report text
    return f"""📋 Onboarding Customer Experience Report

Email Validation Status:
----------------------
{email_validation}

Bottleneck Analysis:
--------------------------
{bottleneck_result}

Activation Status:
------------------
{activation_result}

Churn Rate Results:
---------------------
{churn_risk_result}

📝 Recommendations:
-------------------
{recommendation_result}

Regards,
AI Reporting Agent
"""

In [ ]:
import requests
import json
from agents import Agent, Runner, function_tool # Import Agent and Runner

# Step 1: Run the onboarding analysis
# Example 3: Incomplete onboarding customer
incomplete_customer = CustomerOnboarding(
    customer_id="CUST_003",
    name="Maria",
    surname="Tanova",
    email="akshata.sawant.tech@gmail.com",
    signup_date="2025-06-26T14:00:00",
    activation_date=None,  # Not activated
    onboarding_steps={
        "identity_verification": 15,
        "credit_check": 20,
        "plan_selection": 45  # Stopped here
    },
    failed_steps=["equipment_provisioning"],
    customer_type="individual",
    service_type="internet",
    support_tickets=3
)
result_3 = await run_onboarding_analysis(incomplete_customer)

# Step 2: Synthesize the report using an Agent
# Define a simple agent to run the synthesize_report tool
report_synthesis_agent = Agent(
    name="ReportSynthesisAgent",
    instructions="Synthesize the provided analysis results into a formatted report.",
    model="gpt-4o-mini", # Using a smaller model as it's just tool execution
    tools=[synthesize_report],
    output_type=str
)

# Run the agent to synthesize the report
synthesized_report_result = await Runner.run(
    report_synthesis_agent,
    input="Synthesize the onboarding analysis results.",
    context=result_3 # Pass the analysis results as context
)

synthesized_report = synthesized_report_result.final_output


# Step 3: Prepare data for Zapier webhook
# Get Zapier webhook URL from userdata or hardcode it (replace with your actual URL)
# url = userdata.get('ZAPIER_WEBHOOK_URL') # Uncomment and use this if storing URL in userdata
url = "https://hooks.zapier.com/hooks/catch/23653760/u34cigo/" # Replace with your actual Zapier webhook URL

# Data to send to Zapier - customize the keys and values as needed for your Zap
payload = {
    "subject": f"Onboarding Analysis Report for {incomplete_customer.name} {incomplete_customer.surname}",
    "recipient_name": "Full-Code Group 2", # Or a specific email address
    "customer_id": incomplete_customer.customer_id,
    "customer_name": f"{incomplete_customer.name} {incomplete_customer.surname}",
    "customer_email": incomplete_customer.email,
    "report_content": synthesized_report
}

# Step 4: Send the data to Zapier webhook using requests
try:
    response = requests.post(url, json=payload)
    response.raise_for_status() # Raise an exception for bad status codes (4xx or 5xx)
    print("📧 Email data sent to Zapier successfully!")
    print("Zapier response:", response.text)

except requests.exceptions.RequestException as e:
    print(f"🚨 Failed to send email data to Zapier: {e}")
    if response:
        print("Zapier response:", response.text)

# The rest of the agent definition and running logic using MCPServerSse is not needed for sending to a standard webhook
# You can remove or comment out the following lines if you only need to send data via requests
# # Step 2: Initialize the MCPServerSse
# mcp_server = MCPServerSse(
#     name="MCP_Email",
#     params={
#         "url": url
#     },
#     client_session_timeout_seconds=30
# )

# # Step 3: Define the agent that uses the MCP server
# report_agent = Agent(
#     name="EmailAgent",
#     instructions="Generate a professional email summary by calling synthesize_report tool, based on analysis received. Recipient name is: Full-Code Group 2. Do not include visuals in emails.",
#     mcp_servers=[mcp_server],
#     model_settings=ModelSettings(tool_choice="required"),
#     model="gpt-4o-mini",
#     tools=[synthesize_report],
#     output_type=str,
# )

# # Step 4: Store the analysis result
# analysis_result = result_3

# # Step 5: Enter MCP server context
# await mcp_server.__aenter__()

# try:
#     # Step 6: Run the agent with the analysis result
#     result = await Runner.run(
#         report_agent,
#         context=analysis_result
#     )
#     print("📧 Email result:", result.final_output)

# finally:
#     # Step 7: Clean up the MCP server session
#     await mcp_server.__aexit__(None, None, None)

📊 Analyzing onboarding for: Maria Tanova (ID: CUST_003)
Email: akshata.sawant.tech@gmail.com | Service: internet | Type: individual
🎯 Auto-calculated Churn Risk: 87.0%
--------------------------------------------------------------------------------
📧 Step 1: Email Validation Agent | Email Validation...
	The validated email address is **akshata.sawant.tech@gmail.com**. 

- **Validation Status**: Valid
- **Confidence Level**: High
- **Issue**: The email does not match the name Maria Tanova.

If you have a different email address for validation or need further assistance, let me know!

🔍 Step 2: Bottleneck Agent | Bottleneck Analysis...
	The analysis of Maria Tanova's onboarding process revealed the following bottlenecks and failed steps:

### Bottlenecks:
- **Identity Verification:** Took 15 minutes, exceeding the expected 10-minute duration.
- **Credit Check:** Took 20 minutes, exceeding the expected 15-minute duration.
- **Plan Selection:** Took 45 minutes, much longer than the expecte